In [2]:
# 固定图像保存

from PIL import Image
import os

def crop_image_to_tiles_and_save(image_path, tile_width, save_dir):
    # 确保保存目录存在
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # 打开图像
    image = Image.open(image_path)
    image_width, image_height = image.size

    # 计算中心点
    center_x, center_y = image_width // 2, image_height // 2

    # 计算开始裁剪的左上角坐标
    start_x = center_x - (tile_width * 2.5)
    start_y = center_y - (tile_width * 2.5)

    for i in range(5):
        for j in range(5):
            # 计算每个小块的左上角和右下角坐标
            left = start_x + j * tile_width
            top = start_y + i * tile_width
            right = left + tile_width
            bottom = top + tile_width

            # 裁剪每个小块
            tile = image.crop((left, top, right, bottom))

            # 构建每个小块的保存路径
            tile_filename = f"tile_{i}_{j}.jpg"
            tile_path = os.path.join(save_dir, tile_filename)

            # 保存每个小块
            tile.save(tile_path)

# 使用示例
crop_image_to_tiles_and_save("G:/CT/1/Reconstruction/1-_IR_rec00000125.bmp", 256, "F:/cement/ct-datasets")  # 替换为您的图像路径和保存目录


In [5]:
# 在区域内部随机保存，并且不能大于某个阈值
import random
from PIL import Image
import os

def random_crop_from_center_and_save(image_path, square_size, crop_size, n, overlap_threshold, save_dir, prefix=""):
    image = Image.open(image_path)
    width, height = image.size

    center_x, center_y = width // 2, height // 2
    square_left = center_x - square_size // 2
    square_top = center_y - square_size // 2
    square_right = center_x + square_size // 2
    square_bottom = center_y + square_size // 2

    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    used_areas = []

    for i in range(n):
        attempts = 0
        while True:
            crop_left = random.randint(square_left, square_right - crop_size)
            crop_top = random.randint(square_top, square_bottom - crop_size)
            crop_rect = (crop_left, crop_top, crop_left + crop_size, crop_top + crop_size)

            overlap = any(
                not (rect[2] + overlap_threshold <= crop_rect[0] or 
                     rect[0] >= crop_rect[2] + overlap_threshold or
                     rect[3] + overlap_threshold <= crop_rect[1] or 
                     rect[1] >= crop_rect[3] + overlap_threshold)
                for rect in used_areas
            )

            if not overlap or attempts > 50:
                break
            attempts += 1
        used_areas.append(crop_rect)
        cropped_image = image.crop(crop_rect)
        save_path = os.path.join(save_dir, f'{prefix}_cropped_{i}.jpg')
        cropped_image.save(save_path)

# 示例使用
random_crop_from_center_and_save("G:/CT/1/Reconstruction/1-_IR_rec00000125.bmp", 256*5, 256, 5, 10, "F:/cement/ct-datasets")
random_crop_from_center_and_save("G:/CT/1/Reconstruction/1-_IR_rec00000126.bmp", 256*5, 256, 5, 10, "F:/cement/ct-datasets", "next")
